In [1]:
#TODO
#put in the code that made 'uniqueDirections' so that it's available
#see what objects you can delete so that this whole thing goes faster
#triple check that interpolation is working on multiple bike rides at once...
#try to increase resolution of video
#try and make a timer that shows what time it is during these frames... maybe a title?
#TODO reexamine why you chose these dates... make sure you get all the rides on these dates (i.e. if we're trying to get 8/9, make sure we have rides that went from 8/8 to 8/9, and 8/9 into 8/10)
#we either need to change the code such that phantomJS and ffmpeg are called directly or add them to the path


#if we really want, maybe we can see if we can improve the bike interpolation
#make this multithreaded so it goes faster?

In [3]:
import polyline

In [4]:
import googlemaps
import json

import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from os.path import isdir
import os

import matplotlib.pyplot as plt
import seaborn as sns
import math

import folium

import subprocess

sns.set_style("darkgrid")

pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns",100)

In [80]:
def print_frames(df_final_rounded_time, date, pandas_date, alternate_range=None):
    directory = 'Image Pipe//' + date + '//'
    if not(os.path.isdir(directory)):
         os.makedirs(directory)
    if alternate_range is None:
        dayInterval = pd.date_range(start=pandas_date, end=pandas_date + pd.DateOffset(1), freq='15s')
    else:
        dayInterval = alternate_range
    i = 0
    for interval in dayInterval:
        #get rows that coincide with the time we're trying to plot
        df_final_rounded_time_temp = df_final_rounded_time[df_final_rounded_time['Rounded Time']==interval]
        #of those, get seperate rows where the bikes are finishing a ride, starting a ride, or in the middle of a ride
        df_final_rounded_time_temp_start = df_final_rounded_time_temp[df_final_rounded_time_temp['Percent Complete']==0]
        df_final_rounded_time_temp_end = df_final_rounded_time_temp[df_final_rounded_time_temp['Percent Complete']==1]
        df_final_rounded_time_temp_middle = df_final_rounded_time_temp[(df_final_rounded_time_temp['Percent Complete']!=0) & (df_final_rounded_time_temp['Percent Complete']!=1)]
        sg_map = folium.Map(location=[40.73, -73.97], tiles='Stamen Terrain', zoom_start=12)
        #define how to deal with bikes who are in each of the three stages of a ride
        def plotDotStart(point):
            '''input: series that contains a numeric named latitude and a numeric named longitude
            this function creates a CircleMarker and adds it to your this_map'''
            folium.CircleMarker(location=[point['Point Latitude'], point['Point Longitude']],
                                fill_color='red', radius=5, popup=str(point['Point Latitude']) + " " + str(point['Point Longitude']),
                                weight=0, fill=True, fill_opacity=.3).add_to(sg_map)
        def plotDotEnd(point):
            '''input: series that contains a numeric named latitude and a numeric named longitude
            this function creates a CircleMarker and adds it to your this_map'''
            folium.CircleMarker(location=[point['Point Latitude'], point['Point Longitude']],
                                fill_color='green', radius=5, popup=str(point['Point Latitude']) + " " + str(point['Point Longitude']),
                                weight=0, fill=True, fill_opacity=.3).add_to(sg_map)
        def plotDotMiddle(point):
            '''input: series that contains a numeric named latitude and a numeric named longitude
            this function creates a CircleMarker and adds it to your this_map'''
            folium.CircleMarker(location=[point['Point Latitude'], point['Point Longitude']],
                                fill_color='yellow', radius=5, popup=str(point['Point Latitude']) + " " + str(point['Point Longitude']),
                                weight=0, fill=True, fill_opacity=.3).add_to(sg_map)
        #apply appropriate plotting function to each dataframe
        df_final_rounded_time_temp_start.apply(plotDotStart, axis = 1)
        df_final_rounded_time_temp_end.apply(plotDotEnd, axis = 1)
        df_final_rounded_time_temp_middle.apply(plotDotMiddle, axis = 1)
        #save the map as html
        sg_map.save(directory + str(i) + '.html')
        i = i + 1

In [6]:
def html_to_png(date, rangeLength=5761):
    #run program from the command line to convert html to png files
    #cmdstringPhantom = (["phantomjs.exe", "convertToPNG-auto.js"])
    cmdstringPhantom = (["C://Users//Bryan//Documents//GitHub//Springboard//Capstone Project//Image Pipe//phantomjs.exe", "convertToPNG-auto.js"])
    myCWD = "C://Users//Bryan//Documents//GitHub//Springboard//Capstone Project//Image Pipe//" + date

    for i in range(rangeLength):    
        javaScriptCode = """var page = require('webpage').create();
        //viewportSize being the actual size of the headless browser
        page.viewportSize = { width: 1024, height: 768 };
        //the clipRect is the portion of the page you are taking a screenshot of
        page.clipRect = { top: 0, left: 0, width: 1024, height: 768 };
        //the rest of the code is the same as the previous example
        page.open('""" + str(i) + """.html', function() {
          page.render('""" + str(i) + """.png');
          phantom.exit();
        });\n"""

        target = open(myCWD + '//convertToPNG-auto.js', 'w')
        target.write(javaScriptCode)
        target.close()
        subprocess.check_call(cmdstringPhantom, stdin=subprocess.PIPE, cwd=myCWD)

In [37]:
def make_video(date):
    #run program from the command convert png files into an animation
    # ffmpeg -f image2 -i %d.png -r 2 mybike.avi
    myCWD = "C://Users//Bryan//Documents//GitHub//Springboard//Capstone Project//Image Pipe//" + date
    #cmdstringFFMPEG = (["C:\\Users\\Bryan\\Documents\\GitHub\\Springboard\\Capstone Project\\Image Pipe\\FFMPEG.exe", "-f", "image2", "-i", "%d.png", "-r", "2", "mybike.avi"])
    videoString = pd.to_datetime(date).strftime('%Y%m%d') + ".avi"
    cmdstringFFMPEG = (["FFMPEG.exe", "-f", "image2", "-i", "%d.png", "-r", "2", videoString])
    subprocess.check_call(cmdstringFFMPEG, stdin=subprocess.PIPE, cwd=myCWD)

In [ ]:
rangeLength=5761

In [8]:
#find and pickle directions between all stations

#prepare data for loading
mypath = "C:\Users\Bryan\Documents\GitHub\Springboard\Capstone Project\Data\\"
dataFiles = [f for f in listdir(mypath) if isfile(join(mypath,f))]
dataFrames = []

#the names of the columns, but not the actual data, change from file to file, to establish
#the names of the columns we want
inputColumns = ['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID', 'Start Station Name', 'Start Station Latitude',
                'Start Station Longitude', 'End Station ID', 'End Station Name', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type', 'Birth Year', 'Gender']

#use to help the read_csv file determine datatypes
inputDtypeDict = {'Trip Duration': np.int64,
                  #'starttime': np.datetime64,
                  #'stoptime': np.datetime64,
                  'Start Station ID': np.int64,
                  'Start Station Name': np.string_,
                  'Start Station Latitude': np.float64,
                  'Start Station Longitude': np.float64,
                  'End Station ID': np.int64,
                  'End Station Name': np.string_,
                  'End Station Latitude': np.float64,
                  'End Station Longitude': np.float64,
                  'Bike ID': np.int64,
                  'User Type': np.string_,
                  'Birth Year': np.string_,
                  'Gender': np.int64,
                  'Start Next Station Id': np.int64
                  }

In [9]:
print dataFiles

['2013-07 - Citi Bike trip data.csv', '2013-08 - Citi Bike trip data.csv', '2013-09 - Citi Bike trip data.csv', '2013-10 - Citi Bike trip data.csv', '2013-11 - Citi Bike trip data.csv', '2013-12 - Citi Bike trip data.csv', '2014-01 - Citi Bike trip data.csv', '2014-02 - Citi Bike trip data.csv', '2014-03 - Citi Bike trip data.csv', '2014-04 - Citi Bike trip data.csv', '2014-05 - Citi Bike trip data.csv', '2014-06 - Citi Bike trip data.csv', '2014-07 - Citi Bike trip data.csv', '2014-08 - Citi Bike trip data.csv', '201409-citibike-tripdata.csv', '201410-citibike-tripdata.csv', '201411-citibike-tripdata.csv', '201412-citibike-tripdata.csv', '201501-citibike-tripdata.csv', '201502-citibike-tripdata.csv', '201503-citibike-tripdata.csv', '201504-citibike-tripdata.csv', '201505-citibike-tripdata.csv', '201506-citibike-tripdata.csv', '201507-citibike-tripdata.csv', '201508-citibike-tripdata.csv', '201509-citibike-tripdata.csv', '201510-citibike-tripdata.csv', '201511-citibike-tripdata.csv', '

In [10]:
#load data into n dataframes
for f in dataFiles:
    data_df = pd.read_csv(mypath + f, header=0, names=inputColumns, dtype=inputDtypeDict, parse_dates=['Start Time', 'Stop Time'], infer_datetime_format=True)
    print "File " + f + " has " + str(len(data_df)) + " records"
    dataFrames.append(data_df)

File 2013-07 - Citi Bike trip data.csv has 843416 records
File 2013-08 - Citi Bike trip data.csv has 1001958 records
File 2013-09 - Citi Bike trip data.csv has 1034359 records
File 2013-10 - Citi Bike trip data.csv has 1037712 records
File 2013-11 - Citi Bike trip data.csv has 675774 records
File 2013-12 - Citi Bike trip data.csv has 443966 records
File 2014-01 - Citi Bike trip data.csv has 300400 records
File 2014-02 - Citi Bike trip data.csv has 224736 records
File 2014-03 - Citi Bike trip data.csv has 439117 records
File 2014-04 - Citi Bike trip data.csv has 670780 records
File 2014-05 - Citi Bike trip data.csv has 866117 records
File 2014-06 - Citi Bike trip data.csv has 936880 records
File 2014-07 - Citi Bike trip data.csv has 968842 records
File 2014-08 - Citi Bike trip data.csv has 963489 records
File 201409-citibike-tripdata.csv has 953887 records
File 201410-citibike-tripdata.csv has 828711 records
File 201411-citibike-tripdata.csv has 529188 records
File 201412-citibike-tripd

In [11]:
#join all dataframes into single dataframe
allTrips_df = pd.concat(dataFrames)
print "Final file has " + str(len(allTrips_df)) + " records"

Final file has 45453878 records


In [12]:
allTrips_df['Start Date'] = allTrips_df['Start Time'].dt.date
allTrips_df['Start Date'] = pd.to_datetime(allTrips_df['Start Date'])

In [13]:
uniqueDirections_df_part1 = pd.read_pickle('additional_trips1.pickle')
uniqueDirections_df_part2 = pd.read_pickle('additional_trips2.pickle')
uniqueDirections_df_part3 = pd.read_pickle('additional_trips3.pickle')
uniqueDirections_df = pd.concat([uniqueDirections_df_part1, uniqueDirections_df_part2, uniqueDirections_df_part3])
uniqueDirections_df.rename(columns={'point latitude': 'Point Latitude',
                                    'point longitude': 'Point Longitude',
                                    'start station latitude': 'Start Station Latitude',
                                    'start station longitude': 'Start Station Longitude',
                                    'end station latitude': 'End Station Latitude',
                                    'end station longitude': 'End Station Longitude'}, inplace=True)

In [14]:
#get rid of duplicate points on same trip... if you redump the data you can solve this problem ahead of time
uniqueDirections_df = uniqueDirections_df.groupby(['Point Latitude', 
                             'Point Longitude', 
                             'Start Station Latitude', 
                             'Start Station Longitude', 
                             'End Station Latitude', 
                             'End Station Longitude'])['order'].min().to_frame().reset_index().\
                                sort_values(['Start Station Latitude',
                               'Start Station Longitude',
                               'End Station Latitude',
                               'End Station Longitude',
                               'order']).reset_index().drop('index', axis=1)

In [15]:
uniqueDirections_df.head()

,Point Latitude,Point Longitude,Start Station Latitude,Start Station Longitude,End Station Latitude,End Station Longitude,order
0,40.646301,-74.016839,40.646538,-74.016588,40.646538,-74.016588,0
1,40.646811,-74.016169,40.646678,-74.016303,40.646678,-74.016303,0
2,40.646920,-74.016350,40.646768,-74.016510,40.646768,-74.016510,0
3,40.646920,-74.016350,40.646768,-74.016510,40.668546,-73.993333,0
4,40.643988,-74.011482,40.646768,-74.016510,40.668546,-73.993333,1


In [16]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [17]:
uniqueDirections_df['dist'] = \
    haversine_np(uniqueDirections_df['Point Longitude'].shift(), uniqueDirections_df['Point Latitude'].shift(),
                 uniqueDirections_df.loc[1:, 'Point Longitude'], uniqueDirections_df.loc[1:, 'Point Latitude'])

In [18]:
uniqueDirections_df['dist'] = np.where(uniqueDirections_df['order']==0, 0, uniqueDirections_df['dist'])

In [19]:
uniqueDirections_df['cumdist'] = uniqueDirections_df.groupby(['Start Station Latitude', 
                  'Start Station Longitude', 
                  'End Station Latitude', 
                  'End Station Longitude'])['dist'].cumsum()

In [20]:
totalTravelDistance = uniqueDirections_df.groupby(['Start Station Latitude', 
                  'Start Station Longitude', 
                  'End Station Latitude', 
                  'End Station Longitude'])['cumdist'].max().to_frame().reset_index()

In [21]:
#get rid of routes that start and end in the same place
totalTravelDistance = totalTravelDistance[totalTravelDistance['cumdist']!=0]

In [22]:
totalTravelDistance.rename(columns={'cumdist':'Total Route Distance'}, inplace=True)

In [23]:
uniqueDirections_df = pd.merge(uniqueDirections_df, totalTravelDistance, how='inner', 
                               on=['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude'])

In [24]:
uniqueDirections_df['Percent Complete'] = uniqueDirections_df['cumdist'] / uniqueDirections_df['Total Route Distance']

In [25]:
#get rid of points that are extremely close together (as they mess with interpolation later if they still exist)
uniqueDirections_df = uniqueDirections_df[~((uniqueDirections_df['dist']< 1.0e-12) & (uniqueDirections_df['dist']!= 0))]

In [26]:
#let's examine only some specific dates that we can later compare with each other
allTrips = allTrips_df[(allTrips_df['Start Date']=='2013-08-08') |
                       (allTrips_df['Start Date']=='2013-08-09') |
                       (allTrips_df['Start Date']=='2013-08-10') |
                       (allTrips_df['Start Date']=='2013-11-07') |
                       (allTrips_df['Start Date']=='2013-11-08') |
                       (allTrips_df['Start Date']=='2013-11-09') |
                       (allTrips_df['Start Date']=='2014-02-06') |
                       (allTrips_df['Start Date']=='2014-02-07') |
                       (allTrips_df['Start Date']=='2014-02-08') |
                       (allTrips_df['Start Date']=='2014-05-08') |
                       (allTrips_df['Start Date']=='2014-05-09') |
                       (allTrips_df['Start Date']=='2014-05-10') |
                       (allTrips_df['Start Date']=='2014-08-07') |
                       (allTrips_df['Start Date']=='2014-08-08') |
                       (allTrips_df['Start Date']=='2014-08-09') |
                       (allTrips_df['Start Date']=='2014-11-06') |
                       (allTrips_df['Start Date']=='2014-11-07') |
                       (allTrips_df['Start Date']=='2014-11-08') |
                       (allTrips_df['Start Date']=='2015-02-05') |
                       (allTrips_df['Start Date']=='2015-02-06') |
                       (allTrips_df['Start Date']=='2015-02-07') |
                       (allTrips_df['Start Date']=='2015-05-07') |
                       (allTrips_df['Start Date']=='2015-05-08') |
                       (allTrips_df['Start Date']=='2015-05-09') |
                       (allTrips_df['Start Date']=='2015-08-06') |
                       (allTrips_df['Start Date']=='2015-08-07') |
                       (allTrips_df['Start Date']=='2015-08-08') |
                       (allTrips_df['Start Date']=='2015-11-05') |
                       (allTrips_df['Start Date']=='2015-11-06') |
                       (allTrips_df['Start Date']=='2015-11-07') |
                       (allTrips_df['Start Date']=='2016-02-04') |
                       (allTrips_df['Start Date']=='2016-02-05') |
                       (allTrips_df['Start Date']=='2016-02-06') |
                       (allTrips_df['Start Date']=='2016-05-05') |
                       (allTrips_df['Start Date']=='2016-05-06') |
                       (allTrips_df['Start Date']=='2016-05-07') |
                       (allTrips_df['Start Date']=='2016-08-11') |
                       (allTrips_df['Start Date']=='2016-08-12') |
                       (allTrips_df['Start Date']=='2016-08-13') |
                       (allTrips_df['Start Date']=='2016-11-03') |
                       (allTrips_df['Start Date']=='2016-11-04') |
                       (allTrips_df['Start Date']=='2016-11-05') |
                       (allTrips_df['Start Date']=='2017-02-02') |
                       (allTrips_df['Start Date']=='2017-02-03') |
                       (allTrips_df['Start Date']=='2017-02-04')]

#TODO reexamine why you chose these dates... make sure you get all the rides on these dates (i.e. if we're trying to get 8/9, make sure we have rides that went from 8/8 to 8/9, and 8/9 into 8/10)
datesList = ['2013-08-09', '2013-08-10', '2013-11-08', '2013-11-09', '2014-02-07', '2014-02-08', '2014-05-09', '2014-05-10', '2014-08-08', '2014-08-09', '2014-11-07', '2014-11-08', '2015-02-06', '2015-02-07', '2015-05-08', '2015-05-09', '2015-08-07', '2015-08-08', '2015-11-06', '2015-11-07', '2016-02-05', '2016-02-06', '2016-05-06', '2016-05-07', '2016-08-12', '2016-08-13', '2016-11-04', '2016-11-05', '2017-02-03', '2017-02-04']
#datesList = ['2013-08-10', '2013-11-08', '2013-11-09', '2014-02-07', '2014-02-08', '2014-05-09', '2014-05-10', '2014-08-08', '2014-08-09', '2014-11-07', '2014-11-08', '2015-02-06', '2015-02-07', '2015-05-08', '2015-05-09', '2015-08-07', '2015-08-08', '2015-11-06', '2015-11-07', '2016-02-05', '2016-02-06', '2016-05-06', '2016-05-07', '2016-08-12', '2016-08-13', '2016-11-04', '2016-11-05', '2017-02-03', '2017-02-04']

In [39]:
#this is where I start having problems...
print "Here we are..."
rangeLength=5761
datesList = ['2013-08-09', '2013-08-10', '2013-11-08']

Here we are...


In [40]:
for date in datesList:
    pandas_date = pd.to_datetime(date)
    alternate_range = pd.date_range(pandas_date, periods=rangeLength, freq='15s')
    allTripsOneDay = allTrips_df[(allTrips_df['Start Date']==pandas_date) | (allTrips_df['Start Date']==(pandas_date - pd.DateOffset(1)))]
    allTripsOneDay = allTripsOneDay.reset_index().drop('index', axis=1).reset_index()
    allTripsOneDay.rename(columns={'index':'factor'}, inplace=True)
    allTripsOneDay['factor'] = allTripsOneDay['factor'] * 2
    df_merge = pd.merge(uniqueDirections_df, allTripsOneDay, how='inner', on=['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude'])
    df_merge.sort_values(['Bike ID', 'Start Time', 'order'], inplace=True)
    df_merge['Time'] = np.where(df_merge['order']==0, df_merge['Start Time'], pd.NaT)
    df_merge['Time'] = pd.to_datetime(df_merge['Time'])
    df_merge['Time'] = np.where(df_merge['Percent Complete']==1, df_merge['Stop Time'], df_merge['Time'])
    df_merge['Time'] = (df_merge['Time'] - np.datetime64('2013-01-01')) / np.timedelta64(1, 's')
    df_merge['interp index'] = df_merge['Percent Complete'] + df_merge['factor']
    df_merge.set_index('interp index', inplace=True)
    df_merge['Time'] = df_merge.groupby(['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 
                  'Start Time', 'Stop Time', 'Bike ID'])[['Time']].apply(pd.DataFrame.interpolate, method='index')['Time']
    df_merge['Time'] = df_merge['Time'] * np.timedelta64(1, 's') + np.datetime64('2013-01-01T00:00:01')
    #make 30 second buckets with min
    df_merge['Rounded Time'] = df_merge['Time'].dt.round('15s')
    # df_merge.to_csv('temp6.csv')
    df_merge_temp = df_merge[['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Time', 'Rounded Time', 'order']]
    df_min_times = df_merge_temp[['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Time', 'Rounded Time']].groupby(['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time'])['Time'].min().to_frame().reset_index()
    df_merge_temp2 = pd.merge(df_merge_temp, df_min_times, on=['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time'])
    df_merge_temp3 = df_merge_temp2.groupby(['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time'])['order'].min().to_frame().reset_index()
    df_merge_min = pd.merge(df_merge_temp3, df_merge_temp2, how='inner', on=['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time', 'order'])
    df_end_points = df_merge[df_merge['Percent Complete']==1].copy()[['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Time', 'Rounded Time', 'order']]
    df_min_with_end_points = pd.concat([df_merge_min, df_end_points])
    df_min_with_max_end_points = df_min_with_end_points[['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time']].groupby(['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time'])['Time'].max().to_frame().reset_index()
    df_min_with_max_end_points2 = pd.merge(df_min_with_end_points, df_min_with_max_end_points, how='inner', on=['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time'])
    df_min_with_max_end_points3 = df_min_with_max_end_points2.groupby(['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time'])['order'].max().to_frame().reset_index()
    df_min_with_max_end_points4 = pd.merge(df_min_with_max_end_points3, df_min_with_max_end_points2, how='inner', on=['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time', 'order'])
    df_final_rounded_time = pd.merge(df_min_with_max_end_points4, df_merge, how='inner', on=['Start Station Latitude', 'Start Station Longitude', 'End Station Latitude', 'End Station Longitude', 'Bike ID', 'Rounded Time', 'Time', 'order'])
    df_final_rounded_time['Transit Date'] = df_final_rounded_time['Rounded Time'].dt.date
    df_final_rounded_time['Transit Date'] = pd.to_datetime(df_final_rounded_time['Transit Date'])
    df_final_rounded_time = df_final_rounded_time[df_final_rounded_time['Transit Date']==date]
    print_frames(df_final_rounded_time, date, pandas_date, alternate_range)
    html_to_png(date, rangeLength)
    make_video(date)

In [ ]:
#write code for rebalancing, repeat above

In [ ]:
#isolate records that indicate that a rebalancing occured
#allTrips_df.sort_values(['bikeid', 'starttime'], inplace=True)
#allTrips_df['start next station id'] = allTrips_df['start station id'].shift(-1)
#rebalancing_df = allTrips_df[allTrips_df['end station id'] != allTrips_df['start next station id']].copy()